#Necessary Imports

In [ ]:
from ast import Pass
import math
from itertools import combinations
from math import isclose
import pandas as pd

In [ ]:
pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import wordnet
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import wordnet, pos_tag
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from ir_explain.pairwise.axioms import pairwise

In [ ]:
from ir_explain.pairwise.explain_more import explain_more

Showcase some features of the ir_explain.pairwise framework example data:

*   TREC-DL-2019




In [ ]:
!pip install ir_datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=564cc2feeae7864339bdfb90b0ca7b318b3ef5a4c874021f07d4466036fcfbd1
  Stored in directory: /root/.cache/pip/wheels/1a/d7/91/7ffb991df87e62355d945745035470ba2616aa3d83a250b5f9
  Created wheel for zlib-state: filename=zlib_state-0.1.6-cp310-cp310-linux_x86_64.whl size=21162 sha256=f4a185e461116f2392914ef5b564

Get some query from the list to test and display the text of the query

In [ ]:
dataset = ir_datasets.load("msmarco-document/trec-dl-2019")
query_id = "183378"

query_text = None
for query in dataset.queries_iter():
    if query.query_id == query_id:
        query_text = query.text
        break
if query_text is not None:
    print(f"Query ID: {query_id}\nQuery Text: {query_text}")
else:
    print(f"No query found with ID: {query_id}")


[INFO] [starting] https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
[INFO] [finished] https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz: [00:00] [4.28kB] [16.2MB/s]
                                                                                                                   

Query ID: 183378
Query Text: exons definition biology


Get the top ranked documents for the given query

In [ ]:
import ir_datasets

dataset = ir_datasets.load("msmarco-document/trec-dl-2019")

target_query_id = "183378"

target_query_docs = []

for scoreddoc in dataset.scoreddocs_iter():
    if scoreddoc.query_id == target_query_id:
        target_query_docs.append((scoreddoc.doc_id, scoreddoc.score))
i=1
for doc_id, score in target_query_docs:
    print(f"Rank: {i}, Doc ID: {doc_id}, Score: {score}")
    i+=1


[INFO] [starting] https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-doctest2019-top100.gz
[INFO] [finished] https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-doctest2019-top100.gz: [00:00] [220kB] [1.10MB/s]


Rank: 1, Doc ID: D1077802, Score: -6.03738
Rank: 2, Doc ID: D2319972, Score: -6.19266
Rank: 3, Doc ID: D3291537, Score: -6.2536
Rank: 4, Doc ID: D359891, Score: -6.39307
Rank: 5, Doc ID: D1806793, Score: -6.53852
Rank: 6, Doc ID: D674120, Score: -6.61844
Rank: 7, Doc ID: D3291538, Score: -6.66812
Rank: 8, Doc ID: D1626054, Score: -6.6795
Rank: 9, Doc ID: D3291534, Score: -6.68517
Rank: 10, Doc ID: D1796583, Score: -6.70215
Rank: 11, Doc ID: D3546110, Score: -6.7995
Rank: 12, Doc ID: D906730, Score: -6.90626
Rank: 13, Doc ID: D3291533, Score: -6.90913
Rank: 14, Doc ID: D2646768, Score: -6.93343
Rank: 15, Doc ID: D3555880, Score: -6.93369
Rank: 16, Doc ID: D2790851, Score: -6.9459
Rank: 17, Doc ID: D2319971, Score: -6.94791
Rank: 18, Doc ID: D2657361, Score: -6.97759
Rank: 19, Doc ID: D1012311, Score: -7.01531
Rank: 20, Doc ID: D2108995, Score: -7.03303
Rank: 21, Doc ID: D3380457, Score: -7.03347
Rank: 22, Doc ID: D1806795, Score: -7.07862
Rank: 23, Doc ID: D2970614, Score: -7.07915
Rank

Lets take the top ranked document and test it against a lower ranked document

In [ ]:
target_doc_id = "D1077802"

doc1 = ""

for doc in dataset.docs_iter():
    if doc.doc_id == target_doc_id:
        doc1 = doc.body
        break

if doc1 is not None:
    print(f"Body of Document ID {target_doc_id}:\n{doc1}")
else:
    print(f"No document found with ID {target_doc_id}")


[INFO] If you have a local copy of https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-docs.trec.gz, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/d4863e4f342982b51b9a8fc668b2d0c0
[INFO] [starting] https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-docs.trec.gz
[INFO] [finished] https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-docs.trec.gz: [02:24] [8.50GB] [58.8MB/s]


Body of Document ID D1077802:
Definition references  (+ images)Wikipedia:
Exonany nucleotide sequence encoded by a gene that remains present within the final mature RNA product of that gene after introns have been removed by RNA splicing.
The term exon refers to both the DNA sequence within a gene and to the corresponding...  (43 of 1099 words)en.wikipedia.org/wiki/Exon  [cite]Exon (disambiguation)may refer to: Exon, a region of DNA that is represented in the mature form of RNA | Exoniensis or Exon., the Post-Nominal Letters for alumni / degrees from the University of Exeter | Exon can also refer to the signature of the Bishop of Exeter | J...  (45 of 126 words, 7 definitions)en.wikipedia.org/wiki/Exon (disambiguation)  [cite]Collins Dictionary:exon | exonic  [adjective, derived] [British] one of the four officers who command the Yeomen of the Guard | any segment of a discontinuous gene the segments of which are separated by ...  (26 of 299 words, 2 definitions, 1 usage example, pronun

In [ ]:
target_doc_id = "D1806793"

doc2 = ""

for doc in dataset.docs_iter():
    if doc.doc_id == target_doc_id:
        doc2 = doc.body
        break

if doc2 is not None:
    print(f"Body of Document ID {target_doc_id}:\n{doc2}")
else:
    print(f"No document found with ID {target_doc_id}")


Body of Document ID D1806793:
Skip to main page content
HOME ABOUT ARCHIVE SUBMIT SUBSCRIBE ADVERTISE AUTHOR INFO CONTACT HELPSearch for Keyword: GO Advanced Search
What is a gene, post-ENCODE?
History and updated definition
Mark B. Gerstein 1, 2, 3, 9, Can Bruce 2, 4, Joel S. Rozowsky 2, Deyou Zheng 2,Jiang Du 3, Jan O. Korbel 2, 5, Olof Emanuelsson 6, Zhengdong D. Zhang 2,Sherman Weissman 7, and Michael Snyder 2, 8+ Author Affiliations1Program in Computational Biology & Bioinformatics, Yale University, New Haven, Connecticut 06511, USA;2Molecular Biophysics & Biochemistry Department, Yale University, New Haven, Connecticut 06511, USA;3Computer Science Department, Yale University, New Haven, Connecticut 06511, USA;4Center for Medical Informatics, Yale University, New Haven, Connecticut 06511, USA;5European Molecular Biology Laboratory, 69117 Heidelberg, Germany;6Stockholm Bioinformatics Center, Albanova University Center, Stockholm University, SE-10691 Stockholm, Sweden;7Genetics Depa

In [ ]:
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_string(input_string):

    words = word_tokenize(input_string)

    processed_words = [ps.stem(word) for word in words if word.lower() not in stop_words]

    return ' '.join(processed_words)


# Preprocess the strings
doc1 = preprocess_string(doc1)
doc2 = preprocess_string(doc2)
query_text = preprocess_string(query_text)



#Applying Selected Axioms to see which doc is preffered


*   TFC1: assigns higher scores
to documents that contain a query term more often normalised by document length

*   LB1:If there is a query term t with tf(t, d1) = 0
and tf(t, d2) > 0, then d1 < d2.

*   LNC1:  two documents are compared that have the same term frequency
for all query terms (10%-softening). Then the shorter document is preferred.

*   AND: from a document pair d1 and d2 where only d1 contains all
query terms, d1 is preferred.

*   PROX1-3: The closer the query terms appear to each other in a document, the more relevant the document is.








In [ ]:
index_path = ""

# Create an instance of the pairwise class
pairwise_instance = pairwise(query_text, doc1, doc2, index_path)

axiom_classes = ["TFC1", "LB1", "LNC1", "AND", " PROX1+PROX2+PROX3 "]

pairwise_instance.explain(query_text, doc1, doc2, axiom_classes)

,Query,Document 1,Document 2,TFC1,LB1,LNC1,AND,PROX1+PROX2+PROX3
0,exon definit biolog,definit refer ( + imag ),skip main page content ho,1,0,1,1,0


More details on the following axiom -


*   TFC1
*   LNC1



In [ ]:
axiom_name = "TFC1"
explanation = pairwise_instance.explain_details(query_text, doc1, doc2, axiom_name)
print(explanation)

Normalised tf score in doc1: 0.014354066985645933
Normalised tf score in doc2: 0.0012175324675324675
None


In [ ]:
axiom_name = "LNC1"
explanation = pairwise_instance.explain_details(query_text, doc1, doc2, axiom_name)
print(explanation)

Number of query terms document 1: 3
Number of query terms document 2: 3

Length of document1: 3806
Length of document2: 62188
None
